### <center> VOTING RULES IN PYTHON </center>
#### <center> MASTER BDMA - DECISION MODELING </center>
##### <center> Authors: Dilbar Isakova, MD Kamrul Islam </center>
##### <center> Professor: Brice Mayag </center>
###### <center> CentraleSupeléc, Fall, 2024 </center>

## Problem Statement

Let us consider an election with *n* voters (0 ≤ n ≤ 200) and *m* candidates (0 ≤ m ≤ 20). We assume that:

- The preferences of each voter are given as a linear order (total order) on the set of candidates.
- All the preferences (of the *n* voters) are contained in an Excel file or a csv file.

This work aims at computing in python language the voting rules introduced in Chapter 2. You can use the examples of this chapter to test your functionalities, especially the following example where we have *m = 4* candidates {a, b, c, d} and *n = 27* voters:

| Number of Voters | Preference Order           |
|------------------|----------------------------|
| 5 voters         | a > b > c > d              |
| 4 voters         | a > c > b > d              |
| 2 voters         | d > b > a > c              |
| 6 voters         | d > b > c > a              |
| 8 voters         | c > b > a > d              |
| 2 voters         | d > c > b > a              |

For the questions 2, 3, 4, and 5, you will have to manage yourself any possible ties among the winners.


#### Generate the data in csv format and read it for further analysis 

In [3]:
import csv
from collections import Counter

def generate_voting_csv(file_path):
    voting_data = [
        (5, ['a', 'b', 'c', 'd']),
        (4, ['a', 'c', 'b', 'd']),
        (2, ['d', 'b', 'a', 'c']),
        (6, ['d', 'b', 'c', 'a']),
        (8, ['c', 'b', 'a', 'd']),
        (2, ['d', 'c', 'b', 'a'])
    ]
    
    with open(file_path, mode='w', newline='') as file:
        csv_writer = csv.writer(file)

        csv_writer.writerow(['Voter Count', '1st Choice', '2nd Choice', '3rd Choice', '4th Choice'])

        for num_voters, preference in voting_data:
            csv_writer.writerow([num_voters] + preference)


file_path = 'voter_preferences.csv'
generate_voting_csv(file_path)


def read_voting_data(file_path):
    voting_data = []
    
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for row in csv_reader:
            if row:
                num_voters = int(row[0])
                preferences = row[1:]
                voting_data.append((num_voters, preferences))
    
    return voting_data

### Question 01
1. Compute a function Plurality returning the result of a plurality voting.

In [4]:
def plurality_voting(file_path):
    """
    Implements the plurality voting rule using the voting data format.
    The candidate with the most first-choice votes is the winner.
    """
    voting_data = read_voting_data(file_path)
    
    first_choice_votes = Counter()
    for num_voters, preferences in voting_data:
        first_choice = preferences[0] 
        first_choice_votes[first_choice] += num_voters
    
    winner = max(first_choice_votes, key=first_choice_votes.get)
    
    return winner, dict(first_choice_votes)

winner, vote_count = plurality_voting(file_path)

print(f"Winner of the election (Plurality): {winner}")
print("Vote Tally:")
for candidate, votes in vote_count.items():
    print(f"{candidate}: {votes} votes")

Winner of the election (Plurality): d
Vote Tally:
a: 9 votes
d: 10 votes
c: 8 votes


### Question 02
2. Compute a function PluralityRunoff returning the result of a plurality Runoff voting (plurality with two
rounds).

In [5]:
def plurality_runoff_voting(file_path):
    """
    Implements the plurality runoff voting rule using the optimized voting data format.
    The candidate with the most first-choice votes wins if they get more than 50% of the votes in the first round.
    If no candidate has more than 50%, a runoff is held between the top two candidates.
    """
    voting_data = read_voting_data(file_path)

    first_choice_votes = Counter()
    total_votes = 0
    
    # Tally the first-choice votes based on the number of voters and their preferences
    for num_voters, preferences in voting_data:
        first_choice = preferences[0]
        first_choice_votes[first_choice] += num_voters
        total_votes += num_voters
    
    # Check if any candidate has more than 50% of the total votes in the first round
    for candidate, votes in first_choice_votes.items():
        if votes > total_votes / 2:
            return candidate, dict(first_choice_votes),
    
    # If no one wins in the first round, we proceed to a runoff between the top two candidates
    top_two_candidates = first_choice_votes.most_common(2)
    candidate_1, candidate_2 = top_two_candidates[0][0], top_two_candidates[1][0]
    
    runoff_votes = {candidate_1: 0, candidate_2: 0}
    
    # Count votes for the top two candidates in the second round based on preferences
    for num_voters, preferences in voting_data:
        if preferences[0] == candidate_1 or preferences[0] == candidate_2:
            # If the first choice is one of the top two candidates, keep it
            runoff_votes[preferences[0]] += num_voters
        else:
            # If the first choice is not in the top two, use the highest-ranked candidate from the two
            for preference in preferences:
                if preference == candidate_1 or preference == candidate_2:
                    runoff_votes[preference] += num_voters
                    break
    
    runoff_winner = max(runoff_votes, key=runoff_votes.get)
    
    return runoff_winner, dict(first_choice_votes), dict(runoff_votes)


runoff_winner, first_round_results, second_round_results = plurality_runoff_voting(file_path)

print(f"Winner of the election (Plurality Runoff): {runoff_winner}")
print("First Round Vote Tally:")
for candidate, votes in first_round_results.items():
    print(f"{candidate}: {votes} votes")

if second_round_results:
    print("\nRunoff Round Vote Tally:")
    for candidate, votes in second_round_results.items():
        print(f"{candidate}: {votes} votes")


Winner of the election (Plurality Runoff): a
First Round Vote Tally:
a: 9 votes
d: 10 votes
c: 8 votes

Runoff Round Vote Tally:
d: 10 votes
a: 17 votes


### Question 03
3. Compute a function CondorcetVoting returning the result of the application of the Condorcet principle (the existence of the Condorcet winner).

In [6]:
def condorcet_voting(file_path):
    """
    Implements the Condorcet voting rule using the optimized voting data format.
    A Condorcet winner is a candidate who wins against every other candidate in pairwise comparisons.
    """
    voting_data = read_voting_data(file_path)
    
    # Get the list of all candidates
    all_candidates = set()
    for _, preferences in voting_data:
        all_candidates.update(preferences)
    
    all_candidates = list(all_candidates)
    
    # Initialize pairwise comparison matrix
    pairwise_comparisons = {candidate: {other: 0 for other in all_candidates if other != candidate} for candidate in all_candidates}
    
    # Conduct pairwise comparisons
    for num_voters, preferences in voting_data:
        for i, candidate in enumerate(preferences):
            for other in preferences[i + 1:]:
                pairwise_comparisons[candidate][other] += num_voters
    
    # Check if there is a Condorcet winner
    condorcet_winner = None
    for candidate in all_candidates:
        is_winner = True
        for other in all_candidates:
            if candidate != other and pairwise_comparisons[candidate][other] <= pairwise_comparisons[other][candidate]:
                is_winner = False
                break
        if is_winner:
            condorcet_winner = candidate
            break
    
    return condorcet_winner, pairwise_comparisons


condorcet_winner, pairwise_results = condorcet_voting(file_path)

if condorcet_winner:
    print(f"Condorcet Winner: {condorcet_winner}")
else:
    print("No Condorcet Winner")

print("\nPairwise Comparison Results:")
for candidate, comparisons in pairwise_results.items():
    for other, votes in comparisons.items():
        print(f"{candidate} vs {other}: {votes} votes")

Condorcet Winner: c

Pairwise Comparison Results:
c vs b: 14 votes
c vs a: 16 votes
c vs d: 17 votes
b vs c: 13 votes
b vs a: 18 votes
b vs d: 17 votes
a vs c: 11 votes
a vs b: 9 votes
a vs d: 17 votes
d vs c: 10 votes
d vs b: 10 votes
d vs a: 10 votes


### Question 04
4. Compute a function BordaVoting returning the result of the application of the Borda principle.

In [7]:
def borda_voting(file_path):
    """
    Implements the Borda voting rule using the optimized voting data format.
    Each candidate is assigned points based on their rank in each voter's preference. 
    The candidate with the highest total score is the winner.
    """
    
    voting_data = read_voting_data(file_path)
    
    # Get the list of all candidates
    all_candidates = set()
    for _, preferences in voting_data:
        all_candidates.update(preferences)
    
    all_candidates = list(all_candidates)
    
    # Initialize Borda scores for each candidate
    borda_scores = {candidate: 0 for candidate in all_candidates}
    
    # Assign points based on rank
    num_candidates = len(all_candidates)
    
    for num_voters, preferences in voting_data:
        for i, candidate in enumerate(preferences):
            # Assign points inversely proportional to the rank (n-1 points for first, n-2 for second, ..., 0 for last)
            borda_scores[candidate] += num_voters * (num_candidates - 1 - i)
    
    # Find the candidate with the highest Borda score
    winner = max(borda_scores, key=borda_scores.get)
    
    return winner, borda_scores


borda_winner, borda_scores = borda_voting(file_path)

print(f"Winner of the election (Borda Voting): {borda_winner}")
print("Borda Scores:")
for candidate, score in borda_scores.items():
    print(f"{candidate}: {score} points")

Winner of the election (Borda Voting): b
Borda Scores:
c: 47 points
b: 48 points
a: 37 points
d: 30 points


In [8]:
def check_conditions(voters):
    total_voters = sum(voters.values())
    candidate_first_choice_count = {}

    # Count the first choice preferences
    for preference, count in voters.items():
        first_choice = preference.split(' > ')[0]
        if first_choice in candidate_first_choice_count:
            candidate_first_choice_count[first_choice] += count
        else:
            candidate_first_choice_count[first_choice] = count

    # Check for no more than 70% of voters having the same "best candidate"
    max_first_choice_percentage = max(candidate_first_choice_count.values()) / total_voters
    if max_first_choice_percentage > 0.7:
        return False, "More than 70% of voters have the same 'best candidate'."

    # Check for at least 20% having different preferences
    # This is automatically satisfied by the creation of diverse preferences if more than one set exists.
    unique_preferences = len(voters)
    if unique_preferences / total_voters < 0.2:
        return False, "Less than 20% of voters have different preferences."

    return True, "Both conditions are satisfied."

voters_preferences = {
    'a > b > c > d > e > f > g > h': 18,
    'b > a > d > c > h > g > f > e': 12,
    'c > d > a > b > e > f > h > g': 10,
    'd > c > b > a > f > e > h > g': 6,
    'e > f > g > h > a > b > c > d': 5,
    'f > e > h > g > a > d > c > b': 3,
    'g > h > e > f > b > a > d > c': 4,
    'h > g > f > e > d > c > b > a': 2
}

conditions_met, message = check_conditions(voters_preferences)
print(message)

Less than 20% of voters have different preferences.


In [23]:
import random
from collections import Counter

def generate_voter_preferences(n_voters, candidates):
    """
    Generates random voter preferences for a given number of voters and candidates.
    Each voter ranks all candidates.
    """
    return [random.sample(candidates, len(candidates)) for _ in range(n_voters)]

def check_minimum_different_preferences(preferences, min_percentage=0.20):
    """
    Checks if at least a given percentage of voters have different preferences.
    """
    unique_preferences = len(set(tuple(p) for p in preferences))
    return unique_preferences >= len(preferences) * min_percentage

def check_max_same_best_candidate(preferences, max_percentage=0.70):
    """
    Checks if no more than a given percentage of voters have the same first-choice candidate.
    """
    first_choice_counts = Counter(p[0] for p in preferences)
    max_first_choice_count = max(first_choice_counts.values())
    return max_first_choice_count <= len(preferences) * max_percentage

def plurality_voting(preferences):
    first_choice_counts = Counter(p[0] for p in preferences)
    return max(first_choice_counts, key=first_choice_counts.get)

def plurality_runoff_voting(preferences):
    # Assuming a basic runoff system that eliminates the weakest candidate in each round
    candidates = list(set(p[0] for p in preferences))
    while len(candidates) > 1:
        first_choice_counts = Counter(p[0] for p in preferences if p[0] in candidates)
        least_popular = min(first_choice_counts, key=first_choice_counts.get)
        candidates.remove(least_popular)
    return candidates[0]

def condorcet_voting(preferences, candidates):
    candidate_pairs = [(x, y) for x in candidates for y in candidates if x != y]
    wins = {candidate: 0 for candidate in candidates}
    for x, y in candidate_pairs:
        x_wins = sum(1 for p in preferences if p.index(x) < p.index(y))
        if x_wins > len(preferences) / 2:
            wins[x] += 1
    return max(wins, key=wins.get)

def borda_voting(preferences, candidates):
    scores = {candidate: 0 for candidate in candidates}
    for preference in preferences:
        for i, candidate in enumerate(preference):
            scores[candidate] += len(candidates) - i
    return max(scores, key=scores.get)

def validate_data(preferences, candidates):
    """
    Validates an election data by checking if the winner is the same for all four voting methods.
    """
    results = {
        'Plurality': plurality_voting(preferences),
        'Plurality Runoff': plurality_runoff_voting(preferences),
        'Condorcet': condorcet_voting(preferences, candidates),
        'Borda': borda_voting(preferences, candidates)
    }
    same_winner = len(set(results.values())) == 1
    return results, same_winner

n_voters = 60
candidates = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

preferences = generate_voter_preferences(n_voters, candidates)

if check_minimum_different_preferences(preferences) and check_max_same_best_candidate(preferences):
    results, is_consistent = validate_data(preferences, candidates)
    print(f"Election Results: {results}")
    print(f"Same winner in all methods: {is_consistent}")
else:
    print("The example does not satisfy the conditions of the problem.")

Election Results: {'Plurality': 'g', 'Plurality Runoff': 'g', 'Condorcet': 'd', 'Borda': 'c'}
Same winner in all methods: False


In [26]:
import random
from collections import Counter
import pandas as pd

def generate_voter_preferences(n_voters, candidates):
    """
    Generates random voter preferences for a given number of voters and candidates.
    Each voter ranks all candidates.
    """
    return [random.sample(candidates, len(candidates)) for _ in range(n_voters)]

def check_minimum_different_preferences(preferences, min_percentage=0.20):
    """
    Checks if at least a given percentage of voters have different preferences.
    """
    unique_preferences = len(set(tuple(p) for p in preferences))
    return unique_preferences >= len(preferences) * min_percentage

def check_max_same_best_candidate(preferences, max_percentage=0.70):
    """
    Checks if no more than a given percentage of voters have the same first-choice candidate.
    """
    first_choice_counts = Counter(p[0] for p in preferences)
    max_first_choice_count = max(first_choice_counts.values())
    return max_first_choice_count <= len(preferences) * max_percentage

def plurality_voting(preferences):
    first_choice_counts = Counter(p[0] for p in preferences)
    return max(first_choice_counts, key=first_choice_counts.get)

def plurality_runoff_voting(preferences):
    candidates = list(set(p[0] for p in preferences))
    while len(candidates) > 1:
        first_choice_counts = Counter(p[0] for p in preferences if p[0] in candidates)
        least_popular = min(first_choice_counts, key=first_choice_counts.get)
        candidates.remove(least_popular)
    return candidates[0]

def condorcet_voting(preferences, candidates):
    candidate_pairs = [(x, y) for x in candidates for y in candidates if x != y]
    wins = {candidate: 0 for candidate in candidates}
    for x, y in candidate_pairs:
        x_wins = sum(1 for p in preferences if p.index(x) < p.index(y))
        if x_wins > len(preferences) / 2:
            wins[x] += 1
    return max(wins, key=wins.get)

def borda_voting(preferences, candidates):
    scores = {candidate: 0 for candidate in candidates}
    for preference in preferences:
        for i, candidate in enumerate(preference):
            scores[candidate] += len(candidates) - i
    return max(scores, key=scores.get)

def validate_data(preferences, candidates):
    """
    Validates an election data by checking if the winner is the same for all four voting methods.
    """
    results = {
        'Plurality': plurality_voting(preferences),
        'Plurality Runoff': plurality_runoff_voting(preferences),
        'Condorcet': condorcet_voting(preferences, candidates),
        'Borda': borda_voting(preferences, candidates)
    }
    same_winner = len(set(results.values())) == 1
    return results, same_winner

n_voters = 60
candidates = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

preferences = generate_voter_preferences(n_voters, candidates)

# Convert preferences to DataFrame for Excel output
df_preferences = pd.DataFrame(preferences, columns=[f'Rank {i+1}' for i in range(len(candidates))])
df_preferences.to_excel('Voter_Preferences.xlsx', index=False)

if check_minimum_different_preferences(preferences) and check_max_same_best_candidate(preferences):
    results, is_consistent = validate_data(preferences, candidates)
    print(f"Election Results: {results}")
    print(f"Same winner in all methods: {is_consistent}")
    
    # Save results to Excel
    df_results = pd.DataFrame([results])
    df_results['Same Winner All Methods'] = is_consistent
    df_results.to_excel('Election_Results.xlsx', index=False)
else:
    print("The example does not satisfy the conditions of the problem.")

Election Results: {'Plurality': 'c', 'Plurality Runoff': 'g', 'Condorcet': 'g', 'Borda': 'b'}
Same winner in all methods: False
